## Construire un modèle de réseaux de neurones avec Keras

Keras est une bibliothèque "haut niveau" utilisée pour simplifier la description de modèles de réseaux de neurones sur Tensorflow (bibliothèque IA de Google). L'avantage surtout est de pouvoir utiliser des GPU pour accélérer le calcul.

Le travail avec Keras suit un cheminement similaires à celui avec Scikit-Learn, mais il y a quelques différences à retenir.

In [ ]:
import tensorflow as tf
from tensorflow import keras
print (tf.__version__)
keras.__version__

Dans le paragraphe suivant vous avez certainement eu un message d'erreur indiquant que vous n'avez pas des GPU. Dans ce cas, Keras utilisera la CPU de la machine.

## Chargement de données

Tout comme Scikit-Learn, Keras a aussi un ensemble de datasets prêt à utilisation pour des exemples. Dans le cas suivant, nous allons charger le dataset MNIST (écriture à la main) et le séparer en deux groupes : Train et Test. Les données de validation (vérification pendant l'entraînement) seront séparés du groupe Train plus tard.

In [ ]:
from keras.datasets import mnist

#(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
x_train.shape
x_test.shape
y_test.shape

In [ ]:
y_train

Les données de MNIST se présentent sous la forme d'images 28x28 pixels, avec 256 tons de gris. Les labels (`y_train`, par exemple) correspondent aux caractères représentés : les chiffres 0 à 9.

Le paragraphe suivant définit une fonction permettant de visualiser ce dataset.

In [ ]:
def plot_images(x,y=None, indices='all', columns=12, x_size=1, y_size=1,
                colorbar=False, y_pred=None, cm='binary', norm=None, y_padding=0.35, spines_alpha=1,
                fontsize=20, interpolation='lanczos'):
 
    if indices=='all': indices=range(len(x))
    if norm and len(norm) == 2: norm = matplotlib.colors.Normalize(vmin=norm[0], vmax=norm[1])
    draw_labels = (y is not None)
    draw_pred   = (y_pred is not None)
    rows        = math.ceil(len(indices)/columns)
    fig=plt.figure(figsize=(columns*x_size, rows*(y_size+y_padding)))
    n=1
    for i in indices:
        axs=fig.add_subplot(rows, columns, n)
        n+=1
        xx=x[i]
        # ---- Shape is (lx,ly)
        if len(x[i].shape)==2:
            xx=x[i]
        # ---- Shape is (lx,ly,n)
        if len(x[i].shape)==3:
            (lx,ly,lz)=x[i].shape
            if lz==1: 
                xx=x[i].reshape(lx,ly)
            else:
                xx=x[i]
        img=axs.imshow(xx,   cmap = cm, norm=norm, interpolation=interpolation)
#         img=axs.imshow(xx,   cmap = cm, interpolation=interpolation)
        axs.spines['right'].set_visible(True)
        axs.spines['left'].set_visible(True)
        axs.spines['top'].set_visible(True)
        axs.spines['bottom'].set_visible(True)
        axs.spines['right'].set_alpha(spines_alpha)
        axs.spines['left'].set_alpha(spines_alpha)
        axs.spines['top'].set_alpha(spines_alpha)
        axs.spines['bottom'].set_alpha(spines_alpha)
        axs.set_yticks([])
        axs.set_xticks([])
        if draw_labels and not draw_pred:
            axs.set_xlabel(y[i],fontsize=fontsize)
        if draw_labels and draw_pred:
            axs.set_xlabel(y[i],fontsize=fontsize)
        if colorbar:
            fig.colorbar(img,orientation="vertical", shrink=0.65)
    plt.show()
    

Nous allons donc afficher l'un des caractères du groupe Train (celui à la position 27) mais aussi tout la plage entre les données 5 et 41. Remarquez le "label" correspondant sous chaque image.

In [ ]:
plot_images(x_train, y_train, [27],  x_size=5,y_size=5, colorbar=True)
plot_images(x_train, y_train, range(5,41), columns=12)

Les paragraphes suivants font plusieurs opérations afin de préparer les données :

**1 - Reformater les données**

Les images 28x28 sont "applaties" en un seul array unidimensionnel de 784 valeurs

In [ ]:
x_train = x_train.reshape(60000, 784) #  28*28
x_test = x_test.reshape(10000, 784)

**2 - Transformation et normalisation des données**

Les valeurs de base son des entiers entre 0 et 255 pour représenter les 256 tons de gris. La majorité des algorithmes utilisent des valeurs réels, de préférence dans la fourchette 0 à 1 ou -1 à 1. 

Les paragraphes suivantes modifient le type des données (`float32`) puis font une normalisation simple (diviser la valeur par 255). Bien sûr, d'autres méthodes de normalisation plus élaborées sont possibles, mais ça suffit pour l'instant.

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [ ]:
x_train /= 255
x_test /= 255

**3 - Transformer les données catégoriques**
Lorsqu'on a des données catégoriques (texte ou numéros), il faut les transformer afin d'éviter des mauvaises compréhensions de la part de l'algorithme (par exemple, supposer que une classe 2 vient toujours après une classe 1). Dans notre cas, nous allons transformer les classes 0 à 9 en représentations numériques (similaire à HotOneEncoder de Sklearn), afin de rendre indépendantes ces classes.

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train = keras.utils.to_categorical(y_train, num_classes=10)
y_test = keras.utils.to_categorical(y_test, num_classes=10)

In [ ]:
y_train

## La Création d'un modèle

Keras a plusieurs modes permettant la création de modèles de réseaux de neurones. Dans ce cas, nous allons utiliser l'API `Sequential` qui permet de décrire couche par couche du réseau et les empiler (grâce à `add()`). 

Nous allons faire un modèle simple avec des réseaux denses (totalement connectés). La première couche définit la taille de l'entrée (les 784 valeurs reçus du dataset), les autres utilisent par défaut la taille de la sortie de la couche précédente. Egalement, nous indiquons que chaque couche comptera avec 10 neurones.

Finalemen, remarquez qu'on utilise deux types de fonction d'activation, sigmoid et softmax.
Pour simplifier la description, sigmoid donne une probabilité entre 0 et 1, alors que Softmax affiche "1" sur la sortie avec la plus grande probabilité et "0" sur les autres. C'est pour cela qu'on utilise Softmax à la sortie, ça permet d'avoir un résultat plutôt qu'une liste de probabilités.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

# Declaration du modèle en Tensorflow2.0
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(10, activation='sigmoid', input_dim =(784)))
model.add(tf.keras.layers.Dense(10, activation='sigmoid'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))


# résumé du modèle
model.summary()



## Entraînement du modèle

Une fois défini le modèle, il faut l'entraîner avec les données. 
Le paragraphe suivant définit les hyperparamètres du modèle, dont le `batch_size`(taille des sous-ensembles utilisés dans la descente de gradient), le nombre d'epochs (parcours de l'ensemble de données d'entraînement).

L'appel à compile indique aussi qu'on utilise le modèle de descente de gradient SGD (il y a plusieurs), que la métrique utilisée est l'accuracy (métrique qui correspond à (TP+TN)/(TP+TN+FP+FN)), et que la fonction de perte est la `categorical_crossentropy`, une fonction qui compare les probabilités pour des labels catégoriques.

In [ ]:
batch_size = 100
#num_classes = 10
epochs= 50

model.compile(loss='categorical_crossentropy',  optimizer='SGD',  metrics=['accuracy'])


Finalement, on lance l'entraînement. Remarquez aussi qu'on n'a pas crée des données Validation avant, on le fera ici en réservant 10% des données de Train. 

Comme le dataset est simple, on peut faire 50 epoch même sans un GPU.

In [ ]:
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1,verbose=1 )

#verbose: Integer. 0, 1, or 2. Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch.
# Je vous invite à lire la documentation : https://keras.io/models/sequential/

Les paragraphes suivants nous permettent de voir comment le modèle améliore sa performance au fil des epochs

In [ ]:
def plot_history(history, figsize=(8,6), 
                 plot={"Accuracy":['accuracy','val_accuracy'], 'Loss':['loss', 'val_loss']}):
    """
    Show history
    args:
        history: history
        figsize: fig size
        plot: list of data to plot : {<title>:[<metrics>,...], ...}
    """
    fig_id=0
    for title,curves in plot.items():
        plt.figure(figsize=figsize)
        plt.title(title)
        plt.ylabel(title)
        plt.xlabel('Epoch')
        for c in curves:
            plt.plot(history.history[c])
        plt.legend(curves, loc='upper left')
        
        plt.show()

In [ ]:
plot_history(history, figsize=(6,4))

Enfin, on peut estimer la performance du modèle avec les données Test. 

Comparez ces valeur avec ceux de l'entraînement (`val_loss` et `val_accuracy``
 ci-dessus).

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test,verbose=0)

print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

Ces résultats montrent que le modèle se porte un peu moins bien avec de nouvelles données, mais ça reste intéressant. 

## Exercice : 
On a obtenu avec ce modèle basique, un taux d'accuracy d'environ 87%.
- Essayer d'améliorer la performence du modèle, en modifiant les fonctions d'activation, ou/et en n ajoutant le nombre de neurones et des couches intermédiaires.



## Bonus : Quelques exemples de fonctions d'activation de base



### Activation functions

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np


x = np.arange(-6, 6, 0.1)

### Linear  :  Fonction d'activation linéaire

C'est une fonction simple de la forme: f(x) = ax ou f(x) = x. En gros, l'entrée passe à la sortie sans une très grande modification ou alors sans aucune modification. 

In [ ]:
def linear(x):
    a = []
    for item in x:
        a.append(item)
    return a

y = linear(x)

plt.plot(x,y)
plt.grid()
plt.show()


### Sigmoid

En mathématiques, la fonction sigmoïde (dite aussi courbe en S) est définie par :

$$ f(x)=\frac{1}{1 + e^{- x}}$$ pour tout réel x ;


Le but premier de la fonction est de réduire la valeur d'entrée pour la réduire entre 0 et 1. En plus d'exprimer la valeur sous forme de probabilité, si la valeur en entrée est un très grand nombre positif, la fonction convertira cette valeur en une probabilité de 1. A l'inverse, si la valeur en entrée est un très grand nombre négatif, la fonction convertira cette valeur en une probabilité de 0. D'autre part, l'équation de la courbe est telle que, seules les petites valeurs influent réellement sur la variation des valeurs en sortie.

La fonction Sigmoïde a plusieurs défaults:

- Elle n'est pas centrée sur zéro, c'est à dire que des entrées négatives peuvent engendrer des sorties positives.

- Etant assez plate, elle influe assez faiblement sur les neurones par rapport à d'autres fonctions d'activations. Le résultat est souvent très proche de 0 ou de 1 causant la saturation de certains neurones.

- Elle est couteuse en terme de calcul car elle comprend la fonction exponentielle.


In [ ]:
def sigmoid(x):
    a = []
    for item in x:
        a.append(1/( (1+math.exp(-item) * 1))) 
        # ici j'ai pri A = 1 : plus A est grand plus on se rapproche à la fonction echelon ...
        
    return a

y = sigmoid(x)

plt.plot(x,y)
plt.grid()
plt.show()


### Tahn :  Tangente Hyperbolique 

Cette fonction ressemble à la fonction Sigmoïde. La différence avec la fonction Sigmoïde est que la fonction Tanh produit un résultat compris entre -1 et 1. La fonction Tanh est en terme général préférable à la fonction Sigmoïde car elle est centrée sur zéro. Les grandes entrées négatives tendent vers -1 et les grandes entrées positives tendent vers 1.

Mis à part cet avantage, la fonction Tanh possède les mêmes autres inconvénients que la fonction Sigmoïde.

In [ ]:
def tanh(x, derivative=False):
    if (derivative == True):
        return (1 - (x ** 2))
    return np.tanh(x)


y = tanh(x)

plt.plot(x,y)
plt.grid()
plt.show()

### ReLU : Unité de Rectification Linéaire
Pour résoudre le problème de saturation des deux fonctions précédentes (Sigmoïde et Tanh) il existe la fonction ReLU (Unité de Rectification Linéaire). Cette fonction est la plus utilisée.

La fonction ReLU est inteprétée par la formule: f(x) = max(0, x). Si l'entrée est négative la sortie est 0 et si elle est positive, alors la sortie est égale à x. Cette fonction d'activation augmente considérablement la convergence du réseau et ne sature pas.

Mais la fonction ReLU n'est pas parfaite. Si la valeur d'entrée est négative, le neurone reste inactif, ainsi les poids ne sont pas mis à jour et le réseau n’apprend pas.

In [ ]:
def relu(x):
    a = []
    for item in x:
        if item > 0:
            a.append(item)
        else:
            a.append(0)
    return a


y = relu(x)

plt.plot(x,y)
plt.grid()
plt.show()